In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

#needed for jupyter notebook
#https://github.com/tensorflow/tensorflow/issues/24828
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

Num GPUs Available:  3


In [2]:
import os

import numpy as np
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image

MODEL_PATH = "saved_models"
net_file = sorted(os.listdir(MODEL_PATH))[-1] #lowest alphabetically = highest acc
net_path = os.path.join(MODEL_PATH, net_file)
net = load_model(net_path)
print("Loaded:", net_path)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Loaded: saved_models/8799_1575782539_highangle_allclass_RES50.h5


In [40]:
# https://github.com/jkjung-avt/keras-cats-dogs-tutorial/blob/master/predict_resnet50.py

TARGET_SIZE = (224,112) #make sure this matches all_class_Resnet50
IMG_DIR = os.path.join("data","valid")
ALL_CLASSES = sorted(os.listdir(os.path.join("data","valid")))

"""
loops through every class of a dir like data/valid or data/train
"""
for cls in ALL_CLASSES: #class is Python keyword
    print("-"*5,cls,"-"*5,"\n")
    CLASS_DIR = os.path.join(IMG_DIR, cls)
    for filename in os.listdir(CLASS_DIR):
        img_path = os.path.join(CLASS_DIR, filename)

        img = image.load_img(img_path, target_size=TARGET_SIZE)
        if not img: continue
        x = image.img_to_array(img)
        x = preprocess_input(x)
        x = np.expand_dims(x, axis=0) #need to add dim to put into resnet

        pred = net.predict(x)[0]
        top_inds = pred.argsort()[::-1]
 
        # show if prediction less than 90% sure OR class is wrong
        if pred[top_inds[0]] < 0.9 or ALL_CLASSES[top_inds[0]] != cls:     
            print(img_path)
            for i in top_inds[:5]: #only show top five
                print('{}: {}'.format(ALL_CLASSES[i], pred[i]))
            print()
    


----- black_bishop ----- 

data/valid/black_bishop/14_high_angle_8.55.06.jpg
black_queen: 0.4708563983440399
black_bishop: 0.26799672842025757
black_king: 0.11064793169498444
black_rook: 0.0830996111035347
black_pawn: 0.05424753949046135

data/valid/black_bishop/38_high_angle_8.44.02.jpg
black_pawn: 0.9952829480171204
black_queen: 0.003970327787101269
black_bishop: 0.0006509324302896857
empty: 5.95941673964262e-05
black_knight: 2.190104714827612e-05

data/valid/black_bishop/23_high_angle_8.54.20.jpg
black_bishop: 0.7350174784660339
black_knight: 0.12436122447252274
black_queen: 0.10986743122339249
black_rook: 0.022629035636782646
black_pawn: 0.0021946579217910767

----- black_king ----- 

----- black_knight ----- 

data/valid/black_knight/18_real_pos.jpg
black_pawn: 0.8074384927749634
black_queen: 0.13025517761707306
black_knight: 0.03706778585910797
black_bishop: 0.020801443606615067
black_rook: 0.002939700847491622

data/valid/black_knight/45_high_angle_8.55.06.jpg
black_queen: 0.655